![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [15]:
# Import necessary libraries
import pandas as pd
from pandas.api.types import CategoricalDtype
# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [16]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

#Filtering the Dataset

def clean_experience(val):
    if pd.isna(val):
        return None
    val = str(val).strip()
    if val == "<1":
        return 0
    if "+" in val:
        return int(val.replace("+", "").replace(">", ""))
    if ">" in val:
        return int(val.replace(">", ""))
    return int(val)

def clean_company_size(val):
    if pd.isna(val):
        return None
    val = str(val).strip()
    if "+" in val:
        return int(val.replace("+", ""))
    if "-" in val:
        return int(val.split("-")[0])
    if "<" in val:
        return int(val.replace("<", ""))
    return int(val)

ds_jobs_transformed['experience_clean'] = ds_jobs_transformed['experience'].apply(clean_experience)
ds_jobs_transformed['company_size_clean'] = ds_jobs_transformed['company_size'].apply(clean_company_size)

ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience_clean'] >= 10) & (ds_jobs_transformed['company_size_clean'] >= 1000)].copy()

#Type Conversions

for col in ds_jobs_transformed.columns:
    if col in ['experience_clean', 'company_size_clean']:
        continue

    nunique = ds_jobs_transformed[col].nunique(dropna=False)

    if nunique == 2:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category').cat.codes.astype(bool)
    elif pd.api.types.is_integer_dtype(ds_jobs_transformed[col]):
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')
    elif pd.api.types.is_float_dtype(ds_jobs_transformed[col]):
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('float16')
    elif pd.api.types.is_object_dtype(ds_jobs_transformed[col]):
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')

# Handle ordinal categories
edu_order = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
if 'education_level' in ds_jobs_transformed.columns:
    ds_jobs_transformed['education_level'] = pd.Categorical(
        ds_jobs_transformed['education_level'],
        categories = edu_order,
        ordered = True
    )

last_new_job_order = ['never' , '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10+']
if "last_new_job" in ds_jobs_transformed.columns:
    ds_jobs_transformed['last_new_job'] = pd.Categorical(
        ds_jobs_transformed['last_new_job'],
        categories  = last_new_job_order,
        ordered = True
    )

enrolled_uni_order = {
    'no_enrollment' : 0,
    'Part time course': 1,
    'Full time course': 2
}

enrolled_dtype = pd.api.types.CategoricalDtype(
    categories=list(enrolled_uni_order.keys()), 
    ordered=True
)
if 'enrolled_university' in ds_jobs_transformed.columns:
    ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype(enrolled_dtype)

experience_order_numeric = sorted(ds_jobs_transformed['experience_clean'].dropna().unique())
ds_jobs_transformed['experience'] = pd.Categorical(
    ds_jobs_transformed['experience_clean'],
    categories = experience_order_numeric,
    ordered = True
)

company_size_order_dict = {
    1: '<10',
    10: '10-49',
    50: '50-99',
    100: '100-499',
    500: '500-999',
    1000: '1000-4999',
    5000: '5000-9999',
    10000: '10000+'
}

# Keep only sizes that exist in the filtered dataset
company_sizes_existing = sorted(ds_jobs_transformed['company_size_clean'].dropna().unique())
company_size_labels = [company_size_order_dict.get(size, str(size)) for size in company_sizes_existing]

company_size_dtype = pd.api.types.CategoricalDtype(
    categories=company_size_labels,
    ordered=True
)

ds_jobs_transformed['company_size'] = pd.Categorical(
    [company_size_order_dict.get(size, str(size)) for size in ds_jobs_transformed['company_size_clean']],
    dtype=company_size_dtype
)

#Drop the helper columns
ds_jobs_transformed.drop(columns=['experience_clean', 'company_size_clean'], inplace=True)

#Compare Memory Usage
print("Original Memory Usage:")
print(ds_jobs.memory_usage(deep=True), '\n')

print("Tranformed Memory Usage:")
print(ds_jobs_transformed.memory_usage(deep=True), "\n")

print(ds_jobs_transformed.info())

Original Memory Usage:
Index                         128
student_id                 153264
city                      1235888
city_development_index     153264
gender                    1040573
relevant_experience       1527274
enrolled_university       1341257
education_level           1231558
major_discipline          1095945
experience                1121964
company_size              1023519
company_type              1047279
last_new_job              1113264
training_hours             153264
job_change                 153264
dtype: int64 

Tranformed Memory Usage:
Index                     17608
student_id                 8804
city                       9586
city_development_index     4402
gender                     2495
relevant_experience        2201
enrolled_university        2525
education_level            2701
major_discipline           2761
experience                 2589
company_size               2504
company_type               2629
last_new_job               3203
training_ho